<a href="https://colab.research.google.com/github/poorvapuri/UML501/blob/main/Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q1


In [13]:
import numpy as np
from sklearn.metrics import r2_score

np.random.seed(0)

# Generate highly correlated dataset
X = np.random.rand(500,7)
X[:,1:] = X[:,[0]] + np.random.rand(500,6)*0.01
y = 5*X[:,0] + 3*X[:,1] + 2*X[:,2] + np.random.randn(500)

# Add bias column
X = np.c_[np.ones(X.shape[0]), X]

def ridge_gradient_descent(X, y, lr, lmbda, epochs=3000):
    m, n = X.shape
    w = np.zeros(n)

    for _ in range(epochs):
        y_pred = X.dot(w)
        grad = (1/m) * X.T.dot(y_pred - y) + (lmbda/m) * w
        grad[0] -= (lmbda/m) * w[0]  # bias not regularized

        w = w - lr * grad

        # if divergence detected, stop & return None
        if np.isnan(w).any() or np.isinf(w).any():
            return None, None, None

    cost = (1/(2*m))*np.sum((X.dot(w) - y)**2) + (lmbda/(2*m))*np.sum(w[1:]**2)
    return w, cost, r2_score(y, X.dot(w))

learning_rates = [0.0001, 0.001, 0.01, 0.1]
lambdas = [1e-15, 1e-10, 1e-5, 1e-3, 0, 1, 10, 20]

best = None

for lr in learning_rates:
    for lmbda in lambdas:
        w, c, r2 = ridge_gradient_descent(X, y, lr, lmbda)

        if w is None:
            continue

        if (best is None) or (r2 > best[3]):
            best = (lr, lmbda, c, r2)

print(" Best Learning Rate:", best[0])
print(" Best Lambda:", best[1])
print(" Minimum Cost:", best[2])
print(" Maximum R2 Score:", best[3])


 Best Learning Rate: 0.1
 Best Lambda: 1e-15
 Minimum Cost: 0.4780047531549035
 Maximum R2 Score: 0.897330003854884


Q2


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import r2_score

df = pd.read_csv("/content/Hitters.csv").dropna()

X = df.drop("Salary",axis=1)
y = df["Salary"]

cat = X.select_dtypes(include="object").columns
ct = ColumnTransformer([("ohe",OneHotEncoder(drop='first'),cat)],remainder="passthrough")
X = ct.fit_transform(X)

sc = StandardScaler()
X = sc.fit_transform(X)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

lr = LinearRegression().fit(X_train,y_train)
ridge = Ridge(alpha=0.5748).fit(X_train,y_train)
lasso = Lasso(alpha=0.5748).fit(X_train,y_train)

models = {"Linear":lr,"Ridge":ridge,"Lasso":lasso}

for name,m in models.items():
    pred = m.predict(X_test)
    print(name,": R2 =",r2_score(y_test,pred))


Linear : R2 = 0.4166986480172832
Ridge : R2 = 0.4437353233291301
Lasso : R2 = 0.43172177389455013


Q3


In [7]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.metrics import r2_score

boston = fetch_openml(name='boston', version=1, as_frame=True)
X,y = boston.data, boston.target

sc = StandardScaler()
X = sc.fit_transform(X)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

ridge = RidgeCV(alphas=[0.01,0.1,1,10,100]).fit(X_train,y_train)
lasso = LassoCV(cv=5).fit(X_train,y_train)

print("Ridge best alpha:",ridge.alpha_,"R2:",r2_score(y_test,ridge.predict(X_test)))
print("Lasso best alpha:",lasso.alpha_,"R2:",r2_score(y_test,lasso.predict(X_test)))


Ridge best alpha: 1.0 R2: 0.7633890084067051
Lasso best alpha: 0.025342972941781578 R2: 0.7623867996124895


Q4


In [6]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy.special import expit

data = load_iris()
X = data.data
y = data.target
X = np.c_[np.ones(X.shape[0]),X]

def logistic_regression_ovr(X,y,lr=0.01,epochs=2000):
    classes = np.unique(y)
    m,n = X.shape
    W = np.zeros((len(classes),n))
    for idx,c in enumerate(classes):
        yc = (y==c).astype(int)
        w = np.zeros(n)
        for _ in range(epochs):
            pred = expit(X.dot(w))
            grad = (1/m)*X.T.dot(pred-yc)
            w -= lr*grad
        W[idx] = w
    return W

def predict_ovr(W,X):
    probs = expit(X.dot(W.T))
    return np.argmax(probs,axis=1)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)
W = logistic_regression_ovr(X_train,y_train)
pred = predict_ovr(W,X_test)
print("Accuracy:",accuracy_score(y_test,pred))


Accuracy: 0.7666666666666667
